In [47]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:5 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,244 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,266 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,337 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubunt

In [48]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [49]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)



In [50]:
# View schema
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [51]:
# Set data types
home_sales_df = home_sales_df.withColumn('price', home_sales_df['price'].cast('float'))
home_sales_df = home_sales_df.withColumn('date', home_sales_df['date'].cast('date'))
home_sales_df = home_sales_df.withColumn('date_built', home_sales_df['date_built'].cast('date'))
home_sales_df = home_sales_df.withColumn('sqft_living', home_sales_df['sqft_living'].cast('int'))
home_sales_df = home_sales_df.withColumn('view', home_sales_df['view'].cast('int'))

In [52]:
# View updated schema
home_sales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: date (nullable = true)
 |-- price: float (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: integer (nullable = true)



In [53]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView('sales_view')

In [55]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
select
 year(date)
,round(avg(price),2)
from sales_view
where bedrooms = '4'
group by 1
order by 1 desc
""").show()


+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           296363.88|
|      2021|           301819.44|
|      2020|           298353.78|
|      2019|            300263.7|
+----------+--------------------+



In [56]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
select
 year(date_built)
,round(avg(price),2)
from sales_view
where bedrooms = '3'
and bathrooms = '3'
group by 1
order by 1 desc
""").show()


+----------------+--------------------+
|year(date_built)|round(avg(price), 2)|
+----------------+--------------------+
|            2017|           292676.79|
|            2016|           290555.07|
|            2015|            288770.3|
|            2014|           290852.27|
|            2013|           295962.27|
|            2012|           293683.19|
|            2011|           291117.47|
|            2010|           292859.62|
+----------------+--------------------+



In [57]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
select
 year(date_built)
,round(avg(price),2)
from sales_view
where bedrooms = '3'
and bathrooms = '3'
and floors = '2'
and sqft_living >= 2000
group by 1
order by 1 desc
""").show()


+----------------+--------------------+
|year(date_built)|round(avg(price), 2)|
+----------------+--------------------+
|            2017|           280317.58|
|            2016|            293965.1|
|            2015|           297609.97|
|            2014|           298264.72|
|            2013|           303676.79|
|            2012|           307539.97|
|            2011|           276553.81|
|            2010|           285010.22|
+----------------+--------------------+



In [58]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()
spark.sql("""
select
 view
,round(avg(price),2)
from sales_view
where price >= 350000
group by 1
order by 1 desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
| 100|           1026669.5|
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 0.5056054592132568 seconds ---


In [60]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table sales_view')

DataFrame[]

In [62]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales_view')

True

In [64]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
spark.sql("""
select
 view
,round(avg(price),2)
from sales_view
where price >= 350000
group by 1
order by 1 desc
""").show()


print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
| 100|           1026669.5|
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 0.29775500297546387 seconds ---


In [65]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
home_sales_df.write.partitionBy('date_built').mode('overwrite').parquet('home_sales_partitioned')

In [66]:
# 11. Read the parquet formatted data.
p_home_sales_p=spark.read.parquet('home_sales_partitioned')

In [67]:
# 12. Create a temporary table for the parquet data.
p_home_sales_p.createOrReplaceTempView('p_sales_p')

In [78]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

start_time = time.time()
spark.sql("""
select
 view
,round(avg(price),2)
from p_sales_p
where price >= 350000
group by 1
order by 1 desc
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
| 100|           1026669.5|
|  99|          1061201.42|
|  98|          1053739.33|
|  97|          1129040.15|
|  96|          1017815.92|
|  95|           1054325.6|
|  94|           1033536.2|
|  93|          1026006.06|
|  92|           970402.55|
|  91|          1137372.73|
|  90|          1062654.16|
|  89|          1107839.15|
|  88|          1031719.35|
|  87|           1072285.2|
|  86|          1070444.25|
|  85|          1056336.74|
|  84|          1117233.13|
|  83|          1033965.93|
|  82|           1063498.0|
|  81|          1053472.79|
+----+--------------------+
only showing top 20 rows

--- 0.38134074211120605 seconds ---


In [81]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales_view")

DataFrame[]

In [82]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("sales_view"):
  print("a table is till cached")
else:
  print("all clear")


all clear
